In [1]:
import sys

sys.path.append("../")
import penaltyblog as pb

In [2]:
df = pb.footballdata.fetch_data("england", 2020, 0)
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,0,1,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,2020-09-12,17:30,Liverpool,Leeds,4,3,H,3,2,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,2020-09-12,20:00,West Ham,Newcastle,0,2,A,0,0,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,2020-09-13,14:00,West Brom,Leicester,0,3,A,0,0,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97


In [3]:
pb.ratings.massey(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])

,team,rating,offence,defence
0,Man City,1.275,1.486184,-0.211184
1,Man United,0.725,1.238962,-0.513962
2,Liverpool,0.65,1.10424,-0.45424
3,Tottenham,0.575,1.108406,-0.533406
4,Chelsea,0.55,0.832018,-0.282018
5,Leicester,0.45,1.115351,-0.665351
6,Arsenal,0.4,0.757018,-0.357018
7,West Ham,0.375,0.952851,-0.577851
8,Aston Villa,0.225,0.76674,-0.54174
9,Leeds,0.2,0.962573,-0.762573


In [4]:
import numpy as np
import pandas as pd

In [5]:
teams = np.sort(np.unique(np.concatenate([df["HomeTeam"], df["AwayTeam"]])))

In [6]:
n_teams = len(teams)
C = np.zeros([n_teams, n_teams])
b = np.zeros([n_teams])
for _, row in df.iterrows():
    if row["FTHG"] == row["FTAG"]:
        continue
    
    h = np.where(teams == row["HomeTeam"])[0][0]
    a = np.where(teams == row["AwayTeam"])[0][0]

    C[h, a] = C[h, a] - 1
    C[a, h] = C[a, h] - 1
    
    if row["FTHG"] > row["FTAG"]:
        b[h] += 1
        b[a] -= 1
        
    elif row["FTHG"] < row["FTAG"]:
        b[h] -= 1
        b[a] += 1       
        
#     if row["FTR"] == "A":
#         b[h] += 0.333
#         b[a] += 0.333          

np.fill_diagonal(C, np.abs(C.sum(axis=1)) + 2)
b = 1 + (b * 0.5)

In [7]:
r = np.linalg.solve(C, b)

r = pd.DataFrame([teams, r]).T
r.columns = ["team", "rating"]
r = r.sort_values("rating", ascending=False)
r

,team,rating
11,Man City,0.776759
12,Man United,0.729115
10,Liverpool,0.672065
4,Chelsea,0.657836
18,West Ham,0.62241
9,Leicester,0.61429
16,Tottenham,0.594971
0,Arsenal,0.577157
6,Everton,0.548732
1,Aston Villa,0.525975
